In [ ]:
%load_ext autoreload
%autoreload 2
from __future__ import division, print_function, absolute_import
import pickle
import os
import glob
import matplotlib.pyplot as plt
import sys
sys.path.append('..')
from edfreader import read_edf
import numpy as np
from lmfit import  Model, Parameters
%matplotlib inline
import pandas as pd

In [ ]:
!ls ../data/*.asc

In [ ]:
!ls ../parametre/

In [ ]:
sujet = ['s01','s02','s03','s04','s05','s06','s07','s08','s09','s10','s11','s12']
time = ['2017-10-23_100057','2017-09-26_095637',
        '2017-10-03_134421','2017-09-27_161040',
        '2017-10-03_143803','2017-09-28_115250',
        '2017-09-20_151043','2017-10-26_121823',
        '2017-11-08_094717','2017-11-16_153313',
        '2017-11-08_150410','2017-11-17_172706']
colors = ['black', 'dimgrey', 'grey', 'darkgrey', 'silver', 'rosybrown', 'lightcoral', 'indianred', 'firebrick', 'brown', 'darkred', 'red']


In [ ]:
liste = []
for fname in glob.glob('parametre/test/*pkl'):
    a = fname.split('/')[2].split('.')[0].split('_')
    if a[0]=='Fit' :
        liste.append(a)
        #print(a)

param = []
w = 0
for x in range(len(liste)) :
    if liste[x][1] in sujet :
        a = 'parametre/test/param_Fit_%s.pkl'%(liste[x][1])
        with open(a, 'rb') as fichier :
            b = pickle.load(fichier, encoding='latin1')
            param.append(b)
        param[w]['sujet'] = liste[x][1]
        w = w + 1
print(param[0].keys())

In [ ]:
def velocity (x, tau, maxi, start_anti, a_anti, latency, bino) :
    
    a_anti = a_anti/1000 # pour passer de sec à ms
    time = np.arange(len(x))
    vitesse = []
                
    for t in range(len(time)):
        
        if start_anti >= latency :
            if time[t] < latency :
                vitesse.append(0)
            else :
                vitesse.append((bino*2-1)*maxi*(1-np.exp(-1/tau*(time[t]-latency))))
        else :

            if time[t] < start_anti :
                vitesse.append(0)
            else :
                if time[t] < latency :
                    #vitesse.append((bino*2-1)*(time[t]-start_anti)*a_anti)
                    vitesse.append((time[t]-start_anti)*a_anti)
                    x = (time[t]-start_anti)*a_anti
                else :
                    vitesse.append((bino*2-1)*maxi*(1-np.exp(-1/tau*(time[t]-latency)))+x)
    return vitesse

In [ ]:
RMS = pd.DataFrame(index=np.arange(600),columns=sujet)

for x in range(len(sujet)) :
    timeStr = time[x]
    observer = sujet[x]
    mode = 'eyeMvt'
    
    print(observer,timeStr)
    
    file = os.path.join('..','data', mode + '_' + observer + '_' + timeStr + '.pkl')
    with open(file, 'rb') as fichier :
        exp = pickle.load(fichier, encoding='latin1')

    resultats = os.path.join('..','data', mode + '_' + observer + '_' + timeStr + '.asc')
    data = read_edf(resultats, 'TRIALID')

    N_trials = exp['N_trials']
    N_blocks = exp['N_blocks']
    p = exp['p']

    
    for z in range(len(param)) :
        if param[z]['sujet'] == observer :
            tau = param[z]['tau']
            maxi = param[z]['maxi']
            start_anti = param[z]['start_anti']
            a_anti = param[z]['a_anti']
            latency = param[z]['latency']
            #fit = param[z]['fit']
            #print(param[z]['sujet'], len(fit[0][0]))

    block = 0
    trial = 0
    l_rms = []
    for block in range(N_blocks) :
        #l_rms = []
        for trial in range(N_trials) :
            bino=p[trial, block, 0]
            #fit_best = fit[block][trial].best_fit
            
            
            trial_data = trial + N_trials*block
            data_x = data[trial_data]['x']
            trackertime = data[trial_data]['trackertime']

            StimulusOn = data[trial_data]['events']['msg'][10][0]
            StimulusOf = data[trial_data]['events']['msg'][14][0]
            TargetOn = data[trial_data]['events']['msg'][15][0]
            TargetOff = data[trial_data]['events']['msg'][16][0]
            saccades = data[trial_data]['events']['Esac']

            trackertime_0 = data[trial_data]['trackertime'][0]
            
            fit_best = velocity(range(len(trackertime)), tau[block][trial], maxi[block][trial],
                                start_anti[block][trial], a_anti[block][trial], latency[block][trial], bino)
            

            gradient_x = np.gradient(data_x)
            gradient_deg = gradient_x * 1/exp['px_per_deg'] * 1000 # gradient en deg/sec

            #-------------------------------------------------
            # SUPPRESSION DES SACCADES
            #-------------------------------------------------
            gradient_deg_NAN = gradient_deg
            for s in range(len(saccades)) :
                if saccades[s][1]-trackertime_0+15 <= (len(trackertime)) :
                    for x_data in np.arange((saccades[s][0]-trackertime_0-5), (saccades[s][1]-trackertime_0+15)) :
                        gradient_deg_NAN[x_data] = np.nan
                        if x_data < len(fit_best) :
                            fit_best[x_data] = np.nan
                else :
                    for x_data in np.arange((saccades[s][0]-trackertime_0-5), (len(trackertime))) :
                        gradient_deg_NAN[x_data] = np.nan
                        if x_data < len(fit_best) :
                            fit_best[x_data] = np.nan
            #-------------------------------------------------
            # SUPPRESSION DES MICRO SACCADES
            #-------------------------------------------------
            VFAC = 5
            msdx = np.sqrt((np.nanmedian(gradient_deg_NAN**2))-((np.nanmedian(gradient_deg_NAN))**2))
            radiusx = VFAC*msdx

            test = (gradient_deg_NAN/radiusx)**2
            indx = [x for x in range(len(test)) if test[x] > 1] # [expression for element in liste if predicat]

            sac = []
            dur = 1
            debut_sac = 1
            k = 0
            mindur = 8 #ms
            maxdur = 100

            for k in np.arange(1, len(indx)-1, 1) :
                if indx[k+1]-indx[k]==1 :
                    dur = dur + 1;
                else :
                    if dur >= mindur and dur < maxdur :
                        fin_sac = k
                        sac.append([indx[debut_sac], indx[fin_sac]])
                    debut_sac = k+1
                    dur = 1
                k = k + 1
                            
            for s in range(len(sac)) :
                if sac[s][1]+15 <= (len(trackertime)) :
                    for x_data in np.arange((sac[s][0]-5), (sac[s][1]+15)) :
                        gradient_deg_NAN[x_data] = np.nan
                        if x_data < len(fit_best) :
                            fit_best[x_data] = np.nan
                else :
                    for x_data in np.arange((sac[s][0]-5), (len(trackertime))) :
                        gradient_deg_NAN[x_data] = np.nan
                        if x_data < len(fit_best) :
                            fit_best[x_data] = np.nan

            #-------------------------------------------------
            fit_best = np.array(fit_best)
            fit2 = np.ma.masked_array(fit_best[StimulusOf-trackertime_0:], mask=np.isnan(fit_best[StimulusOf-trackertime_0:])).compressed()
            gradient_deg_NAN2 = np.ma.masked_array(gradient_deg_NAN[StimulusOf-trackertime_0:len(fit_best)], mask=np.isnan(fit_best[StimulusOf-trackertime_0:])).compressed()
            a=0
            b=0
            for t in range(len(fit2)) :
                a = a + (gradient_deg_NAN2[t]-fit2[t])**2

            rms = np.sqrt(a)/len(fit2)

            l_rms.append(rms)
            
    RMS[observer] = l_rms

In [ ]:
RMS.mean(0)

In [ ]:
RMS.std(0)

In [ ]:
RMS.max(0)

In [ ]:
fig, ax = plt.subplots(1,len(sujet), figsize=(5*len(sujet), (5*1)/1.6180))
print(len(ax))
a=0
for x in sujet :
    print(x)

    ax[a].hist(RMS[x], color=colors[a], label=x)#, bins=np.arange(0, 1, 0.05))
    ax[a].text(0.25, 150, '%s - %0.4f'%(np.argmax(RMS[x]), np.max(RMS[x])), color='k')
    ax[a].text(0.25, 100, '%s - %0.4f'%(np.argmin(RMS[x]), np.min(RMS[x])), color='k')
    ax[a].legend(ncol=4)
    a=a+1

In [ ]:
s='s05'
for x in range(len(RMS[s])) :
    if RMS[s][x] > 0.45 :
        print(x+1)